In [55]:
import pandas as pd
import numpy as np
import geopandas
%matplotlib inline

In [64]:
rec = pd.read_csv("City of Pittsburgh Playing Fields.csv", index_col = "id")
numrec = rec.groupby('neighborhood').count()
playground = pd.read_csv("Playgrounds.csv")
play = playground.groupby('neighborhood').count()
recreation = pd.concat([numrec, play])
court = pd.read_csv("Courts and Rinks.csv")
court = court.groupby('neighborhood').count()
recreation = pd.concat([recreation, court])['park']
recreation = pd.DataFrame(recreation)
recreation = recreation.groupby('neighborhood').sum()
rec = recreation.sort_values(by=['park'], ascending=False)
rec

,park
neighborhood,
Squirrel Hill South,39
Highland Park,28
Hazelwood,17
South Side Slopes,17
Brookline,17
...,...
Homewood South,1
Homewood North,1
West Oakland,1


In [108]:
parkranks = {}
counter = 0
prev = -1
same = 1
for index,row in rec.iterrows():
    if row['park'] == prev:
        same = same + 1
    else:
        counter = counter + same
        same = 1
    parkranks[index]= counter
    prev = row['park']
parkranks = parkranks.items()
column = ["neighborhood","parkrank"]
parkranks = pd.DataFrame(parkranks, columns=column)
parkranks

,neighborhood,parkrank
0,Squirrel Hill South,1
1,Highland Park,2
2,Hazelwood,3
3,South Side Slopes,3
4,Brookline,3
...,...,...
67,Homewood South,66
68,Homewood North,66
69,West Oakland,66
70,Esplen,66


In [70]:
Arrest = pd.read_csv("Arrest in Pittsburgh.csv")
arrest = Arrest.groupby('INCIDENTNEIGHBORHOOD').count().sort_values(by=['OFFENSES'], ascending=True)

In [71]:
crimeranks = {}
counter = 0
prev = -1
same = 1
for index,row in arrest.iterrows():
    if row['OFFENSES'] == prev:
        same = same + 1
    else:
        counter = counter + same
        same = 1
    crimeranks[index]= counter
    prev = row['OFFENSES']
crimeranks = crimeranks.items()
column = ["neighborhood","crimerank"]
crimeranks = pd.DataFrame(crimeranks, columns=column)
crimeranks

,neighborhood,crimerank
0,Mt. Oliver Neighborhood,1
1,New Homestead,2
2,Swisshelm Park,2
3,Ridgemont,4
4,Chartiers City,5
...,...,...
92,East Liberty,93
93,Carrick,94
94,East Allegheny,95
95,Central Business District,96


In [74]:
Enrollment = pd.read_csv("Enrollment by Neighborhood 2020-21 school yr.csv")
Total_Enrollment_Year = Enrollment.groupby('neighborhood').sum()
tey = Total_Enrollment_Year.sort_values(by=["total_students_enrolled"],ascending=False)
tey = pd.DataFrame(tey)

In [72]:
edranks = {}
counter = 0
prev = -1
same = 1
for index,row in tey.iterrows():
    if row['total_students_enrolled'] == prev:
        same = same + 1
    else:
        counter = counter + same
        same = 1
    edranks[index]= counter
    prev = row['total_students_enrolled']
edranks = edranks.items()
column = ["neighborhood","edrank"]
edranks = pd.DataFrame(edranks, columns=column)
edranks

,neighborhood,edrank
0,Carrick,1
1,Brookline,2
2,Beechview,3
3,Sheraden,4
4,Squirrel Hill South,5
...,...,...
86,North Oakland,83
87,South Shore,83
88,Ridgemont,83
89,Golden Triangle,83


In [86]:
ranking = crimeranks.merge(edranks)
ranking = ranking.merge(parkranks)
ranking.set_index('neighborhood')

,crimerank,edrank,parkrank
neighborhood,,,
New Homestead,2,68,44
Swisshelm Park,2,75,44
East Carnegie,9,82,44
Oakwood,9,67,33
Regent Square,11,77,18
...,...,...,...
Sheraden,92,4,9
East Liberty,93,15,14
Carrick,94,1,14


In [99]:
avgrank = (ranking['crimerank'] + ranking['edrank'] + ranking['parkrank']) / 3
avgrank = pd.DataFrame(avgrank, columns=['avg'])
avgrank = ranking.merge(avgrank, left_index=True, right_index=True)
avg = avgrank.sort_values(by=['avg'],ascending=True)

,neighborhood,crimerank,edrank,parkrank,avg
41,Squirrel Hill South,62,5,1,22.666667
8,Lincoln Place,20,36,22,26.000000
36,Highland Park,56,21,2,26.333333
28,Terrace Village,45,23,22,30.000000
16,Morningside,29,43,18,30.000000
...,...,...,...,...,...
33,West Oakland,52,70,66,62.666667
31,Strip District,50,83,57,63.333333
45,Central Oakland,67,83,44,64.666667
65,Bluff,90,73,66,76.333333
